In [102]:
import numpy as np
import pandas as pd
import scipy.sparse as ssp
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics, model_selection
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_predict,cross_val_score
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
import lightgbm
from lightgbm.sklearn import LGBMClassifier
import os 
import re
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier 

# I will use cross_val_score on XGBoost to select 100,200 or 300 for each preprocess

In [109]:
work_train_w2v = {} 
work_test_w2v = {}
pre_process=["w2v_100.csv","w2v_200.csv","w2v_300.csv"]
path="checkpoints_databases/"
for f in pre_process:
    work_train_w2v[re.sub("\.csv","",f)] = pd.read_csv(path+"w_working_train_"+f)
    work_test_w2v[re.sub("\.csv","",f)] = pd.read_csv(path+"w_working_test_"+f)

In [110]:
work_train_d2v = {} 
work_test_d2v = {}
pre_process=["d2v_100.csv","d2v_200.csv","d2v_300.csv"]
path="checkpoints_databases/"
for f in pre_process:
    work_train_d2v[re.sub("\.csv","",f)] = pd.read_csv(path+"w_working_train_"+f)
    work_test_d2v[re.sub("\.csv","",f)] = pd.read_csv(path+"w_working_test_"+f)

In [111]:
work_train_tfidf = {} 
work_test_tfidf = {}
pre_process=["tfidf_tsvd_100.csv","tfidf_tsvd_200.csv","tfidf_tsvd_300.csv"]
path="checkpoints_databases/"
for f in pre_process:
    work_train_tfidf[re.sub("\.csv","",f)] = pd.read_csv(path+"w_working_train_"+f)
    work_test_tfidf[re.sub("\.csv","",f)] = pd.read_csv(path+"w_working_test_"+f)

In [112]:
new_train_v=pd.read_csv("../bases/new_training_variants.csv",header=None)

In [113]:
y=np.array(new_train_v.iloc[:,0])-1

In [114]:
train = pd.read_csv('../bases/new_training_variants.csv').reset_index()
train.columns=[["Tempo_ID","Class"]]
test = pd.read_csv('../bases/new_test_variants.csv')
ID_train=train.Tempo_ID
ID_test=test.ID
del train,test

In [115]:
kf = model_selection.StratifiedKFold(n_splits=5, random_state=26, shuffle=True)

Start 

In [120]:
clf_xgb=XGBClassifier(max_depth=5, objective="multi:softprob",seed=26)
for name in work_train_w2v:
    h=cross_val_score(clf_xgb,np.array(work_train_w2v[name].drop("ID",axis=1)),y,cv=kf,n_jobs=-1,scoring="neg_log_loss")
    print("mean"+name+" "+str(h.mean()),
         "std:"+name+" "+str(h.std()))
#200 wins

meanw2v_100 -0.9226482822 std:w2v_100 0.0461198601161
meanw2v_200 -0.921885803638 std:w2v_200 0.050552558084
meanw2v_300 -0.925538385872 std:w2v_300 0.0442563156976


In [121]:
for name in work_train_d2v:
    h=cross_val_score(clf_xgb,np.array(work_train_d2v[name].drop("ID",axis=1)),y,cv=kf,n_jobs=-1,scoring="neg_log_loss")
    print("mean"+name+" "+str(h.mean()),
         "std:"+name+" "+str(h.std()))

meand2v_100 -1.0462248149 std:d2v_100 0.0430902269406
meand2v_200 -1.05292029529 std:d2v_200 0.0368450376392
meand2v_300 -1.05027731603 std:d2v_300 0.0412608589716


In [122]:
for name in work_train_tfidf:
    h=cross_val_score(clf_xgb,np.array(work_train_tfidf[name].drop("ID",axis=1)),y,cv=kf,n_jobs=-1,scoring="neg_log_loss")
    print("mean"+name+" "+str(h.mean()),
         "std:"+name+" "+str(h.std()))

meantfidf_tsvd_100 -0.900147502713 std:tfidf_tsvd_100 0.0472139830118
meantfidf_tsvd_200 -0.906877603924 std:tfidf_tsvd_200 0.0525136319305
meantfidf_tsvd_300 -0.920302415821 std:tfidf_tsvd_300 0.0544607246809


# GRID SEARCH PHASE ALL ALGOS

In [123]:
work_train= {} 
work_test = {}
pre_process=["d2v_100.csv","tfidf_tsvd_100.csv","w2v_200.csv"]
path="checkpoints_databases/"
for f in pre_process:
    work_train[re.sub("\.csv","",f)] = pd.read_csv(path+"w_working_train_"+f)
    work_test[re.sub("\.csv","",f)] = pd.read_csv(path+"w_working_test_"+f)

In [ ]:
clf_xgb=XGBClassifier(max_depth=3, objective="multi:softprob",seed=26)
param_test= {
    "max_depth" : [3,5,7],
    "min_child_weight" : [1,3],
    "n_estimators" : [100,200],
    "subsample":[0.8,1],
    "colsample_bytree":[0.8,1]
}
for name in work_train:
    gsearch=GridSearchCV(estimator=clf_xgb, param_grid = param_test,scoring="neg_log_loss",n_jobs=-1,iid=False, cv=kf,verbose=True)
    gsearch.fit(work_train[name],y)
    print(name)
    print(gsearch.grid_scores_)
    print(gsearch.best_params_)
    print(gsearch.best_score_)
#d2v :
#tfidf:
#w2v:

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  6.9min


In [ ]:
clf_lgbm=LGBMClassifier(seed=26)
param_test= {
    'n_estimators': [8,24,48],
    'num_leaves': [6,12,16,22],
    'boosting_type' : ['gbdt'],
    'objective' : ['binary'],
    'colsample_bytree' : [0.7 0.8],
    'subsample' : [0.7,0.8]
    }
for name in work_train:
    gsearch=GridSearchCV(estimator=clf_lgbm, param_grid = param_test,scoring="neg_log_loss",n_jobs=-1,iid=False, cv=kf,verbose=True)
    gsearch.fit(work_train[name],y)
    print(name)
    print(gsearch.grid_scores_)
    print(gsearch.best_params_)
    print(gsearch.best_score_)
#d2v :
#tfidf :
#w2v :

In [ ]:
clf_log=LogisticRegression()
param_test= {
    "C" : [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    "penalty" : ["l1","l2"]
}
for name in work_train:
    gsearch=GridSearchCV(estimator=clf_log, param_grid = param_test,scoring="neg_log_loss",n_jobs=-1,iid=False, cv=kf,verbose=True)
    gsearch.fit(work_train[name],y)
    print(name)
    print(gsearch.grid_scores_)
    print(gsearch.best_params_)
    print(gsearch.best_score_)
#d2v : 
#tfidf :
#w2v :

In [ ]:
clf_ada=AdaBoostClassifier(n_estimators=100, learning_rate=0.3, algorithm="SAMME.R", random_state=26)
param_test={
    "n_estimators":[50,70,100,200],
    "learning_rate":[0.01,0.05,0.1,0.2]
}
for name in work_train:
    gsearch=GridSearchCV(estimator=clf_ada, param_grid = param_test,scoring="neg_log_loss",n_jobs=-1,iid=False, cv=kf,verbose=True)
    gsearch.fit(work_train[name],y)
    print(name)
    print(gsearch.grid_scores_)
    print(gsearch.best_params_)
    print(gsearch.best_score_)
#d2v : 0.3, 50
#tfidf :
#w2v :

In [ ]:
clf_dt=RandomForestClassifier()
param_test={
    "max_depth":[5,7,10,15],
    "n_estimators":[100,200,300]
}
for name in work_train:
    gsearch=GridSearchCV(estimator=clf_dt, param_grid = param_test,scoring="neg_log_loss",n_jobs=-1,iid=False, cv=kf,verbose=True)
    gsearch.fit(work_train[name],y)
    print(name)
    print(gsearch.grid_scores_)
    print(gsearch.best_params_)
    print(gsearch.best_score_)
#d2v : 10 depth,200 estimators
#tfidf : 5, 300estimators
#w2v : 5,300

# TRAINING PHASE ALL ALGOS 

In [8]:
def model_gen(X,X_test,y,classifier,file,five_fold_predict=True):
    #if not os.path.exists("scores/"+file):
    #   os.makedirs("scores/"+file)
    if five_fold_predict:
        fold = 0
        y_test=0
        for train_index, test_index in kf.split(X, y):
        
            fold += 1

            X_train, X_valid    = X[train_index],   X[test_index]
            y_train, y_valid    = y[train_index],   y[test_index]

            print("Fold", fold, X_train.shape, X_valid.shape)

            clf=classifier
            clf.fit(X_train,y_train)
            p_test = clf.predict_proba(X_test)
            y_test += p_test/5

    classes = "class1,class2,class3,class4,class5,class6,class7,class8,class9".split(',')
    subm = pd.DataFrame(y_test, columns=classes)
    subm['ID'] = ID_test
    
    subm.to_csv("w_scores/w_stack_test/w_{}.csv".format(file),index=False)
    
    print("cross_val sur train ") #peut etre que to array est exclusivement pour les xgb
    
    if os.path.isfile("w_scores/w_stack_train/w_{}.csv".format(file)):
        print("not necessary, already done")
    else:
        y_pred=cross_val_predict(estimator=clf,X=X,y=y,cv=kf,method="predict_proba")
        subm1 = pd.DataFrame(y_pred, columns=classes)
        subm1['ID'] = ID_train
        subm1.to_csv("w_scores/w_stack_train/w_{}.csv".format(file),index=False)


In [ ]:
dic_xgb={"xgb_d2v":XGBClassifier(max_depth=3,objective="multi:softprob",seed=26),
        "xgb_tfidf":XGBClassifier(max_depth=3,objective="multi:softprob",seed=26),
        "xgb_w2v":XGBClassifier(max_depth=3,objective="multi:softprob",seed=26)}
dic_lgbm={"lgbm_d2v":LGBMClassifier(seed=26),
        "lgbm_tfidf":LGBMClassifier(seed=26),
        "lgbm_w2v":LGBMClassifier(seed=26)}
dic_lr={"lr_d2v":LogisticRegression(),
        "lr_tfidf":LogisticRegression(),
        "lr_w2v":LogisticRegression()}
dic_ada={"ada_d2v":AdaBoostClassifier(n_estimators=100, learning_rate=1.0, algorithm="SAMME.R", random_state=26),
        "ada_tfidf":AdaBoostClassifier(n_estimators=100, learning_rate=1.0, algorithm="SAMME.R", random_state=26),
        "ada_w2v":AdaBoostClassifier(n_estimators=100, learning_rate=1.0, algorithm="SAMME.R", random_state=26)}
dic_rf={"rf_d2v":RandomForestClassifier(n_estimators=200,max_depth=10,random_state=26),
        "rf_tfidf":RandomForestClassifier(n_estimators=200,max_depth=10,random_state=26),
        "rf_w2v":RandomForestClassifier(n_estimators=200,max_depth=10,random_state=26)}

print("xgboost here")
for clf,name in zip(dic_xgb.keys(),work_train.keys()):
    model_gen(X=work_train[name],X_test=work_test[name],y=y,classifier=dic_xgb[clf],file=clf)
print("lgbm here")
for clf,name in zip(dic_lgbm.keys(),work_train.keys()):
    model_gen(X=work_train[name],X_test=work_test[name],y=y,classifier=dic_lgbm[clf],file=clf)
print("logreg here")
for clf,name in zip(dic_lr.keys(),work_train.keys()):
    model_gen(X=work_train[name],X_test=work_test[name],y=y,classifier=dic_lr[clf],file=clf)
print("adaboost here")
for clf,name in zip(dic_ada.keys(),work_train.keys()):
    model_gen(X=work_train[name],X_test=work_test[name],y=y,classifier=dic_ada[clf],file=clf)
print("random forest here")
for clf,name in zip(dic_rf.keys(),work_train.keys()):
        model_gen(X=work_train[name],X_test=work_test[name],y=y,classifier=dic_rf[clf],file=clf)